In [1]:
from keras.datasets import mnist
from keras import utils as kut
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255
x_test = x_test / 255
y_train = kut.to_categorical(y_train)
y_test = kut.to_categorical(y_test)

In [3]:
def batches(x, y, size):
    num = x.shape[0]
    s, e = 0, size
    idx = np.random.permutation(num)
    while s < num:
        yield x[idx[s:e]], y[idx[s:e]]
        s = e
        e = e + size

In [4]:
def accuracy(y, yhat):
    y = np.apply_along_axis(np.argmax, 1, y)
    yhat = np.apply_along_axis(np.argmax, 1, yhat)
    return np.sum(y == yhat) / len(y)

In [8]:
W1 = tf.Variable(np.random.randn(100, 28*28)*0.1, dtype=tf.float32)
b1 = tf.Variable(np.zeros(100), dtype=tf.float32)
W2 = tf.Variable(np.random.randn(10, 100)*0.1, dtype=tf.float32)
b2 = tf.Variable(np.zeros(10), dtype=tf.float32)


X = tf.placeholder(shape=(None, 28, 28), dtype=tf.float32)
Y = tf.placeholder(shape=(None, 10), dtype=tf.float32)


H0 = tf.layers.Flatten()(X)
H1 = tf.tensordot(H0, W1, axes=(1,1)) + b1
YHAT = tf.tensordot(H1, W2, axes=(1,1)) + b2
print('X:{}\nH0:{}\nH1:{}\nYHAT:{}'.format(X.shape, H0.shape, H1.shape, YHAT.shape))

COST = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=YHAT, labels=Y))
opt = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999).minimize(COST)

X:(?, 28, 28)
H0:(?, 784)
H1:(?, 100)
YHAT:(?, 10)


In [12]:
num_epoch = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epoch):
        print('===={}===='.format(epoch))
        batch = 0
        for batch_x, batch_y in batches(x_train, y_train, 20):
            sess.run(opt, {X:batch_x, Y:batch_y})
            if batch % 500 == 0:
                c_train, yhat_train = sess.run([COST, YHAT], {X:x_train, Y:y_train})
                print("""----{batch}----
train cost = {c_train}
train accuracy = {a_train}
""".format(batch=batch, c_train=c_train,
           a_train=accuracy(y_train, yhat_train)))
            batch += 1

====0====
----0----
train cost = 2.7127816677093506
train accuracy = 0.24448333333333333

----500----
train cost = 0.8517187237739563
train accuracy = 0.7919333333333334

----1000----
train cost = 0.611080527305603
train accuracy = 0.8390166666666666

----1500----
train cost = 0.5998668074607849
train accuracy = 0.8483833333333334

----2000----
train cost = 0.5820091962814331
train accuracy = 0.8720333333333333

----2500----
train cost = 0.5290821194648743
train accuracy = 0.8793666666666666

====1====
----0----
train cost = 0.5752270817756653
train accuracy = 0.85845

----500----
train cost = 0.5720058679580688
train accuracy = 0.8710333333333333

----1000----
train cost = 0.5627877712249756
train accuracy = 0.8838

----1500----
train cost = 0.5560592412948608
train accuracy = 0.8739333333333333

----2000----
train cost = 0.5934527516365051
train accuracy = 0.8673

----2500----
train cost = 0.645536482334137
train accuracy = 0.8585166666666667

====2====
----0----
train cost = 0.48882